# Obteniendo el TOTAL de vacunados y fallecidos por COVID-19 por cada semana epidemiológica de todo el Perú

En este notebook se busca obtener un dataset con el total de **fallecidos** y vacunados **(dosis 1,2 y 3)** por cada **año** y **semana epidemiológica** de todo el Perú. 

Se considera como **completamente vacunado** a todas las personas reciberon 2 dosis de vacunación.

## 0. Cargar librerías

In [2]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
from epiweeks import Week

import functions as fn

## 1. Procesar el dataset de fallecidos

El dataset de fallecidos será procesado con la librería **Pandas**.

### 1.1. Leer y procesar el dataset

Como se sabe que cada fila representa a una persona, solamente se toma la columna **'fecha de fallecimiento'**.

Se añaden 3 columnas. El año, la semana epidemiológica correspondiente y un contador para contar cada caso de fallecimiento por COVID-19. Se elimina la columna **'fecha_fallecimiento'**.

In [3]:
# Leer el dataset
url_fal = 'RawData/fallecidos_covid.csv'
fal_col = ['FECHA_FALLECIMIENTO', 'DEPARTAMENTO', 'EDAD_DECLARADA']
df_fal = pd.read_csv(url_fal, sep = ';', usecols = fal_col, dtype = {'FECHA_FALLECIMIENTO':'int32',
                                                                     'EDAD_DECLARADA': 'int8'})
del url_fal, fal_col

# Transformar a formato fecha y convertir a semana epidemiológia y año
df_fal.loc[:,'FECHA_FALLECIMIENTO'] = pd.to_datetime(df_fal['FECHA_FALLECIMIENTO'], format = '%Y%m%d')
fn.date_to_epiweek(df_fal,'FECHA_FALLECIMIENTO') 
del df_fal['FECHA_FALLECIMIENTO']

#df_fal['SEXO'].replace({'MASCULINO': 0,'FEMENINO':1}, inplace=True)
#df_fal = df_fal[df_fal['SEXO'].apply(lambda x: str(x).isdigit())]

df_fal = df_fal.astype({'year': 'int16', 'epi_week': 'int8'})
df_fal.head()

,EDAD_DECLARADA,DEPARTAMENTO,year,epi_week
0,77,ANCASH,2021,47
1,70,LIMA,2021,17
2,70,LIMA,2021,17
3,59,AYACUCHO,2021,17
4,55,LIMA,2021,16


Se cuenta cada caso de la variable **'fallecidos'** respecto al año y semana epidemiológica. 

In [4]:
# Devuelve un dataframe con el total de FALLECIDOS por año y semana epidemiológica
epi_fal = df_fal.groupby(['year', 'epi_week']).count()
epi_fal = epi_fal.rename(columns = {'DEPARTAMENTO':'total_fallecidos'})
del epi_fal['EDAD_DECLARADA']

epi_fal.head()

total_fallecidos
year epi_week                  
2020 10                       3
     11                       3
     12                      36
     13                      64
     14                     226

Se obtiene un dataframe **'epi_fal'** con el total de fallecidos por año y semana epidemiológica de todo el Perú.

## 2. Procesar el dataset de vacunados

### 2.1. Leer y procesar el dataset

El dataset de vacunados **'TB_VACUNACION_COVID19.csv'** no está incluido en el repositorio de GitHub debido a su excesivo tamaño (~2gb). Sin embargo se puede descargar directamente desde la página de Datos Abiertos de COVID-19 de MINSA, [Vacunación contra la COVID-19](https://www.datosabiertos.gob.pe/dataset/vacunacion) y colocarlo en la dirección RawData.

Debido al tamaño del dataset de vacunados y para preservar recursos, no se procesará la información usando **Pandas**. En su lugar se utilizará el paquete **Dask**. El dataset no será un **pandas dataframe** si no un **dask dataframe** el cual está separado en n particiones de tipo **pandas dataframe**.

In [5]:
vac_url = 'RawData/TB_VACUNACION_COVID19.csv'
vac_col = ['fecha_vacunacion','dosis']
df_vac = dd.read_csv(vac_url, sep = ",", usecols = vac_col, 
                     dtype = {'fecha_vacunacion':'int32', 'dosis': 'int8'})
del vac_col

# Personas con 3 o menos dosis
df_vac = df_vac[df_vac['dosis'] <= 3]

# Convertimos a formato fecha la columna 'fecha_vacunacion'
df_vac = df_vac.assign(fecha_vacunacion = dd.to_datetime(df_vac["fecha_vacunacion"], format = "%Y%m%d", 
                                                         errors="coerce"))

# Obtenemos el año y semana epidemiológica en una sola columna
df_vac['epi_date'] = df_vac['fecha_vacunacion'].map(lambda date : Week.fromdate(date).isoformat())
del df_vac['fecha_vacunacion']

# Se separa la columna obtenida en 2, una para el año y otra para la semana epidemiológica
df_vac[['year','epi_week']] = df_vac['epi_date'].str.split("W", 1, expand=True)
del df_vac['epi_date']

# Cambiamos el tipo de variable para consumir menos recursos
df_vac['year'] = df_vac['year'].astype('int16')
df_vac['epi_week'] = df_vac['epi_week'].astype('int8')

df_vac.head()

,dosis,year,epi_week
0,1,2021,49
1,2,2021,38
2,2,2021,50
3,1,2021,47
4,1,2021,46


### 2.3. Crosstab del número de dosis (1,2 y 3) respecto al año y semana epidemiológica

**Nota:** No se puede usar multi-index en pivot_table para dask, debido a eso se crea la función **'crosstab4dask'**. Aún es necesario buscar en dask funciones para iterar todas las particiones más facilmente y optimizar el script **(Pendiente)**.

In [6]:
def crosstab4dask(ddf):
    """Función que recibe un dask dataframe (ddf) y realiza conteos de variables en cada partición
    y devuelve el total por los index que hayamos ingresado, en este caso 'year' y 'epi_week'."""
    
    lst = []  # Lista para almacenar la sumatoria de cada particion

    for i in range(0, ddf.npartitions):
        ddf = df_vac.partitions[i].compute()
        lst.append(pd.crosstab(index=[ddf['year'],ddf['epi_week']], columns = ddf['dosis']))

    merged_epivac = pd.concat(lst, axis=1)  # Merge all dfs
    del lst

    merged_epivac = merged_epivac.fillna(0).astype(np.int64)
    merged_epivac = merged_epivac.groupby(level=0, axis=1).sum()
    
    return merged_epivac

In [7]:
vac_dose = crosstab4dask(df_vac)
vac_dose.head()

dosis           1   2  3
year epi_week           
2020 18         1   0  0
     37        22   0  0
     38        35   0  0
     39        45   0  0
     40        71  19  0

Se obtiene un dataset **'vac_dose'** con el número de dosis aplicada por año y semana epidemiológica de todo el Perú.

## 4. Procesar el dataset de infectados

### 4.1. Leer y procesar el dataset

In [8]:
cas_url = 'RawData/positivos_covid.csv'
cas_col = ['FECHA_RESULTADO','EDAD']
df_cas = pd.read_csv(cas_url, sep = ';', usecols = cas_col)

df_cas.dropna(inplace=True)
df_cas = df_cas.astype({'EDAD': 'int8', 'FECHA_RESULTADO': 'int32'})

# Agregamos el año y semana epidemilógica de cada fallecido
df_cas.loc[:,'FECHA_RESULTADO'] = pd.to_datetime(df_cas['FECHA_RESULTADO'], format = '%Y%m%d')
fn.date_to_epiweek(df_cas,'FECHA_RESULTADO') 
del df_cas['FECHA_RESULTADO']

#fn.variable_edad(df_cas, 'EDAD')
df_cas.head()

,EDAD,year,epi_week
0,25,2020,51
1,20,2020,34
2,22,2020,31
3,18,2021,26
4,17,2021,14


In [9]:
# Devuelve un dataframe con el total de CASOS por año y semana epidemiológica
epi_cas = df_cas.groupby(['year', 'epi_week']).count()
epi_cas = epi_cas.rename(columns = {'EDAD':'total_casos'})

epi_cas.head()

total_casos
year epi_week             
2020 10                  6
     11                 51
     12                279
     13                458
     14               1993

## 5. Unir los dataframes de fallecidos y dosis de vacuna por año y semana epidemiológica

In [10]:
# Unimos ambos dataframes por semana epidemiológica
epiweeks = pd.concat([epi_cas, epi_fal, vac_dose], axis=1)  

epiweeks = epiweeks.fillna(value = 0) # Rellenamos vacíos o Nan values con 0
del epi_cas, epi_fal, vac_dose, df_vac

# Cambiamos a int ya que existen Nan values y cambia a float64 automáticamente
epiweeks = epiweeks.astype('int64')

# Rename columns
epiweeks.rename(columns={1: 'total_dosis_1', 2: 'total_dosis_2', 3: 'total_dosis_3'}, inplace=True)
                                    
epiweeks.head()

total_casos  total_fallecidos  total_dosis_1  total_dosis_2  \
year epi_week                                                                
2020 10                  6                 3              0              0   
     11                 51                 3              0              0   
     12                279                36              0              0   
     13                458                64              0              0   
     14               1993               226              0              0   

               total_dosis_3  
year epi_week                 
2020 10                    0  
     11                    0  
     12                    0  
     13                    0  
     14                    0

# Obteniendo el número de muertes confirmadas y vacunados contra COVID-19 por semana epidemiológica de cada uno de los 24 departamentos del Perú 

### 2.3. Realizar una crosstab de fallecidos por COVID-19 por departamento de Perú, para cada año y semana epidemiológica

Esto se lo realiza para obtener el TOTAL de fallecidos por cada departamento y semana epidemiológica

In [11]:
falxdep = pd.crosstab(index=[df_fal['year'], df_fal['epi_week']],
                      columns=[df_fal['DEPARTAMENTO']],
                      margins = False)

falxdep = falxdep.add_suffix('_fal')

falxdep.head()

DEPARTAMENTO   AMAZONAS_fal  ANCASH_fal  APURIMAC_fal  AREQUIPA_fal  \
year epi_week                                                         
2020 10                   0           0             0             0   
     11                   0           0             0             0   
     12                   0           0             0             0   
     13                   0           4             0             0   
     14                   0           0             0            15   

DEPARTAMENTO   AYACUCHO_fal  CAJAMARCA_fal  CALLAO_fal  CUSCO_fal  \
year epi_week                                                       
2020 10                   0              0           0          0   
     11                   0              0           0          0   
     12                   1              0           0          1   
     13                   0              0           2          3   
     14                   0              2          15          2   

DEPARTAMENTO   HUANCAVELICA_fal  HUANUCO_fal  ...  LORETO_fal  \
year epi_week                                 ...               
2020 10                       0            0  ...           1   
     11                       0            0  ...           1   
     12                       0            0  ...           3   
     13                       0            0  ...           6   
     14                       0            0  ...           9   

DEPARTAMENTO   MADRE DE DIOS_fal  MOQUEGUA_fal  PASCO_fal  PIURA_fal  \
year epi_week                                                          
2020 10                        0             0          0          0   
     11                        0             0          0          1   
     12                        0             0          0          1   
     13                        1             0          0          2   
     14                        0             0          0          9   

DEPARTAMENTO   PUNO_fal  SAN MARTIN_fal  TACNA_fal  TUMBES_fal  UCAYALI_fal  
year epi_week                                                                
2020 10               0               2          0           0            0  
     11               0               0          0           0            0  
     12               0               0          0           0            2  
     13               1               0          0           0            3  
     14               0               3          1           3           11  

[5 rows x 25 columns]

In [12]:
fn.variable_edad(df_fal, 'EDAD_DECLARADA')
del df_fal['DEPARTAMENTO']

df_fal.head()

,year,epi_week,edad_cat
0,2021,47,6
1,2021,17,6
2,2021,17,6
3,2021,17,4
4,2021,16,4


In [13]:
df_fal_edad = pd.crosstab(index = [df_fal['year'], df_fal['epi_week']], columns= df_fal['edad_cat'])
df_fal_edad = df_fal_edad.rename({0: '0_17_fal', 1:'18_29_fal', 2: '30_39_fal', 3:'40_49_fal', 4: '50_59_fal',
                                  5:'60_69_fal', 6:'70_79_fal', 7:'80_more_fal'}, axis=1)
df_fal_edad.head()

edad_cat       0_17_fal  18_29_fal  30_39_fal  40_49_fal  50_59_fal  \
year epi_week                                                         
2020 10               0          1          0          0          0   
     11               2          0          0          0          0   
     12               0          2          3          6          6   
     13               5          3          4          6          8   
     14               3          5         13         12         47   

edad_cat       60_69_fal  70_79_fal  80_more_fal  
year epi_week                                     
2020 10                1          0            1  
     11                0          1            0  
     12                5          8            6  
     13               12         16           10  
     14               47         51           48

Ahora tenemos el número de fallecidos por cada departamento y semana epidemiológica

## 3. Procesar el dataset de vacunados

No existe una manera directa de calcular el total de personas completamente vacunadas por departamento. Para lograrlo se prevee lo siguiente:

- El dataset de vacunados (RawData/TB_VACUNACION_COVID19.csv) solo proporciona información sobre el centro de vacunación llamado **'id_centro_vacunacion'**. NO el departamento.

- El dataset de los centros de vacunación [(RawData/TB_CENTRO_VACUNACION.csv)]() se puede utilizar para hacer **"match"** del **'id_centro_vacunacion'** del dataset de vacunados con la variable **'id_ubigeo'**. Que es una variable numérica de 0 a 1894 que representa a cada distrito.

- Finalmente, con el dataset de UBIGEO [(RawData/TB_UBIGEOS.csv)]() es posible reemplazar a cada **'id_ubigeo'** con el departamento respectivo.

In [14]:
# Se lee otro dataset porque es más eficiente que separar y crear otro
vac_col = ['fecha_vacunacion','dosis', 'id_eess', 'edad']
df_vac = dd.read_csv(vac_url, sep = ",", usecols = vac_col, 
                     dtype = {'fecha_vacunacion':'int32', 'dosis': 'int8','id_eess':'int32', 'edad': 'float64'})
del vac_col

df_vac = df_vac.dropna() # Drop nan values
df_vac['edad'] = df_vac['edad'].astype('int8')

# Dejar solamente a personas con 2 dosis (será el dataset para el dataset en departamentos)
df_vac = df_vac[df_vac['dosis'] == 2]
del df_vac['dosis']

# Convertimos a formato fecha la columna 'fecha_vacunacion'
df_vac = df_vac.assign(fecha_vacunacion = dd.to_datetime(df_vac["fecha_vacunacion"], format = "%Y%m%d", 
                                                         errors="coerce"))

# Obtenemos el año y semana epidemiológica en una sola columna
df_vac['epi_date'] = df_vac['fecha_vacunacion'].map(lambda date : Week.fromdate(date).isoformat())
del df_vac['fecha_vacunacion']

# Se separa la columna obtenida en 2, una para el año y otra para la semana epidemiológica
df_vac[['year','epi_week']] = df_vac['epi_date'].str.split("W", 1, expand=True)
del df_vac['epi_date']

# Cambiamos el tipo de variable para consumir menos recursos
df_vac['year'] = df_vac['year'].astype('int16')
df_vac['epi_week'] = df_vac['epi_week'].astype('int8')


def edad_cat(df, col):
    """Función para categorizar edad"""

    conditions = [(df[col]< 18),
                  (df[col]>=18) & (df[col]<30),
                  (df[col]>=30) & (df[col]<40),
                  (df[col]>=40) & (df[col]<50),
                  (df[col]>=50) & (df[col]<60),
                  (df[col]>=60) & (df[col]<70),
                  (df[col]>=70) & (df[col]<80),
                  (df[col]>=80)]
    choices = [0,1,2,3,4,5,6,7]

    return np.select(conditions, choices, default=np.nan).astype('int8')

df_vac['edad_cat'] = df_vac.map_partitions(edad_cat, 'edad')
del df_vac['edad']

df_vac.head()

,id_eess,year,epi_week,edad_cat
1,8915,2021,38,2
2,25811,2021,50,0
6,4508,2021,13,1
7,26262,2021,42,1
8,17256,2021,45,1


In [15]:
def crosstab4dask(ddf):
    """Función que recibe un dask dataframe (ddf) y realiza conteos de variables en cada partición
    y devuelve el total por los index que hayamos ingresado, en este caso 'year' y 'epi_week'."""
    
    lst = [] # Lista para almacenar la sumatoria de cada particion

    for i in range(0, ddf.npartitions):
        ddf = df_vac.partitions[i].compute()
        lst.append(pd.crosstab(index=[ddf['year'],ddf['epi_week']], columns = ddf['edad_cat']))

    merged_epivac = pd.concat(lst, axis=1)  # Merge all dfs
    del lst

    merged_epivac = merged_epivac.fillna(0).astype(np.int64)
    merged_epivac = merged_epivac.groupby(level=0, axis=1).sum()
    
    return merged_epivac

In [16]:
df_vac_edad = crosstab4dask(df_vac)
df_vac_edad = df_vac_edad.rename({0: '0_17_vac', 1:'18_29_vac', 2: '30_39_vac', 3:'40_49_vac', 4: '50_59_vac',
                                  5:'60_69_vac', 6:'70_79_vac', 7:'80_more_vac'}, axis=1)
df_vac_edad.head()

edad_cat       0_17_vac  18_29_vac  30_39_vac  40_49_vac  50_59_vac  \
year epi_week                                                         
2020 40               0          7          3          1          3   
     41               0         12         10          4          4   
     42               0         19         13         10          2   
     43               0         26         13          8         16   
     44               0         30         21         14         13   

edad_cat       60_69_vac  70_79_vac  80_more_vac  
year epi_week                                     
2020 40                5          0            0  
     41                1          0            0  
     42                0          0            0  
     43                3          0            0  
     44               15          1            0

### 3.3. Procesar los datasets de **UBIGEO** y **CENTROS DE VACUNACIÓN**.

#### 3.3.1. Leer y procesar los datasets

Para el dataset de UBIGEO solo se necesita las columnas 'id_ubigeo' y 'departamento'. Para el dataset de CENTROS DE VACUNACIÓN solo se necesitan las columnas 'id_centro de vacunación' e 'id_ubigeo'.

In [17]:
ubigeo_url = 'RawData/TB_UBIGEOS.csv'
vaccenter_url = 'RawData/TB_EESS.csv'

# Leemos los datasets de ubigeo y centros de vacunación
ubigeo = pd.read_csv(ubigeo_url, usecols = ['id_ubigeo', 'departamento'])
vaccenter = pd.read_csv(vaccenter_url, usecols= ['id_eess','id_ubigeo'])

# Unimos ambos dataset mediante 'id_ubigeo'
vaccenter = vaccenter.merge(ubigeo, on = 'id_ubigeo', how = 'left')
del ubigeo, vaccenter['id_ubigeo']

vaccenter.head()

,id_eess,departamento
0,5845,LORETO
1,15599,LORETO
2,18282,LORETO
3,18156,LORETO
4,858,LORETO


Ahora con el nuevo dataset **(vaccenter)** es posible encontrar el departamento correspondiente a 'id_centro de vacunacion' del dataset de vacunados **(df_vac)**.

#### 3.3.3. Encontrar el departamento correspondiente de todas las personas vacunadas

In [18]:
df_vac = df_vac.merge(vaccenter, on = 'id_eess', how = 'left')
del df_vac['id_eess']
del vaccenter

df_vac.head()

,year,epi_week,edad_cat,departamento
0,2021,38,2,LIMA
1,2021,50,0,LAMBAYEQUE
2,2021,13,1,LIMA
3,2021,42,1,HUANUCO
4,2021,45,1,LIMA


### 3.4. Crosstab del número de dosis (1,2 y 3) respecto al año y semana epidemiológica

**Nota:** No se puede usar multi-index en pivot_table para dask, debido a eso se crea la función **'crosstab4dask'**. Aún es necesario buscar en dask funciones para iterar todas las particiones más facilmente y optimizar el script **(Pendiente)**.

In [19]:
def crosstab4dask(ddf):
    """Función que recibe un dask dataframe (ddf) y realiza conteos de variables en cada partición
    y devuelve el total por los index que hayamos ingresado, en este caso 'year' y 'epi_week'."""
    
    lst = [] # Lista para almacenar la sumatoria de cada particion

    for i in range(0, ddf.npartitions):
        ddf = df_vac.partitions[i].compute()
        lst.append(pd.crosstab(index=[ddf['year'],ddf['epi_week']], columns = ddf['departamento']))

    merged_epivac = pd.concat(lst, axis=1)  # Merge all dfs
    del lst

    merged_epivac = merged_epivac.fillna(0).astype(np.int64)
    merged_epivac = merged_epivac.groupby(level=0, axis=1).sum()
    
    return merged_epivac

In [20]:
vac_dose = crosstab4dask(df_vac)
del df_vac

vac_dose = vac_dose.add_suffix('_vac')
vac_dose.head()

departamento   AMAZONAS_vac  ANCASH_vac  APURIMAC_vac  AREQUIPA_vac  \
year epi_week                                                         
2020 40                   0           0             0             0   
     41                   0           0             0             0   
     42                   0           0             0             0   
     43                   0           0             0             0   
     44                   0           0             0             0   

departamento   AYACUCHO_vac  CAJAMARCA_vac  CALLAO_vac  CUSCO_vac  \
year epi_week                                                       
2020 40                   0              0           0          0   
     41                   0              0           0          0   
     42                   0              0           0          0   
     43                   0              0           0          0   
     44                   0              0           0          0   

departamento   HUANCAVELICA_vac  HUANUCO_vac  ...  LORETO_vac  \
year epi_week                                 ...               
2020 40                       0            0  ...           0   
     41                       0            0  ...           0   
     42                       0            0  ...           0   
     43                       0            0  ...           0   
     44                       0            0  ...           0   

departamento   MADRE DE DIOS_vac  MOQUEGUA_vac  PASCO_vac  PIURA_vac  \
year epi_week                                                          
2020 40                        0             0          0          0   
     41                        0             0          0          0   
     42                        0             0          0          0   
     43                        0             0          0          0   
     44                        0             0          0          0   

departamento   PUNO_vac  SAN MARTIN_vac  TACNA_vac  TUMBES_vac  UCAYALI_vac  
year epi_week                                                                
2020 40               0               0          0           0            0  
     41               0               0          0           0            0  
     42               0               0          0           0            0  
     43               0               0          0           0            0  
     44               0               0          0           0            0  

[5 rows x 25 columns]

In [21]:
falvac_dep = pd.concat([falxdep, vac_dose], axis = 1).fillna(0)
falvac_dep.head()

AMAZONAS_fal  ANCASH_fal  APURIMAC_fal  AREQUIPA_fal  \
year epi_week                                                         
2020 10                 0.0         0.0           0.0           0.0   
     11                 0.0         0.0           0.0           0.0   
     12                 0.0         0.0           0.0           0.0   
     13                 0.0         4.0           0.0           0.0   
     14                 0.0         0.0           0.0          15.0   

               AYACUCHO_fal  CAJAMARCA_fal  CALLAO_fal  CUSCO_fal  \
year epi_week                                                       
2020 10                 0.0            0.0         0.0        0.0   
     11                 0.0            0.0         0.0        0.0   
     12                 1.0            0.0         0.0        1.0   
     13                 0.0            0.0         2.0        3.0   
     14                 0.0            2.0        15.0        2.0   

               HUANCAVELICA_fal  HUANUCO_fal  ...  LORETO_vac  \
year epi_week                                 ...               
2020 10                     0.0          0.0  ...         0.0   
     11                     0.0          0.0  ...         0.0   
     12                     0.0          0.0  ...         0.0   
     13                     0.0          0.0  ...         0.0   
     14                     0.0          0.0  ...         0.0   

               MADRE DE DIOS_vac  MOQUEGUA_vac  PASCO_vac  PIURA_vac  \
year epi_week                                                          
2020 10                      0.0           0.0        0.0        0.0   
     11                      0.0           0.0        0.0        0.0   
     12                      0.0           0.0        0.0        0.0   
     13                      0.0           0.0        0.0        0.0   
     14                      0.0           0.0        0.0        0.0   

               PUNO_vac  SAN MARTIN_vac  TACNA_vac  TUMBES_vac  UCAYALI_vac  
year epi_week                                                                
2020 10             0.0             0.0        0.0         0.0          0.0  
     11             0.0             0.0        0.0         0.0          0.0  
     12             0.0             0.0        0.0         0.0          0.0  
     13             0.0             0.0        0.0         0.0          0.0  
     14             0.0             0.0        0.0         0.0          0.0  

[5 rows x 50 columns]

# 6. Unir ambos dataframes

In [22]:
epiweeks_final = pd.concat([epiweeks, df_fal_edad, df_vac_edad, falvac_dep, ], axis = 1).fillna(0).astype('int64')
epiweeks_final.head()

total_casos  total_fallecidos  total_dosis_1  total_dosis_2  \
year epi_week                                                                
2020 10                  6                 3              0              0   
     11                 51                 3              0              0   
     12                279                36              0              0   
     13                458                64              0              0   
     14               1993               226              0              0   

               total_dosis_3  0_17_fal  18_29_fal  30_39_fal  40_49_fal  \
year epi_week                                                             
2020 10                    0         0          1          0          0   
     11                    0         2          0          0          0   
     12                    0         0          2          3          6   
     13                    0         5          3          4          6   
     14                    0         3          5         13         12   

               50_59_fal  ...  LORETO_vac  MADRE DE DIOS_vac  MOQUEGUA_vac  \
year epi_week             ...                                                
2020 10                0  ...           0                  0             0   
     11                0  ...           0                  0             0   
     12                6  ...           0                  0             0   
     13                8  ...           0                  0             0   
     14               47  ...           0                  0             0   

               PASCO_vac  PIURA_vac  PUNO_vac  SAN MARTIN_vac  TACNA_vac  \
year epi_week                                                              
2020 10                0          0         0               0          0   
     11                0          0         0               0          0   
     12                0          0         0               0          0   
     13                0          0         0               0          0   
     14                0          0         0               0          0   

               TUMBES_vac  UCAYALI_vac  
year epi_week                           
2020 10                 0            0  
     11                 0            0  
     12                 0            0  
     13                 0            0  
     14                 0            0  

[5 rows x 71 columns]

## 6. Guardar el dataset

In [23]:
epiweeks_final.to_csv('Data/DP1_vacunados_y_fallecidos_x_semanaEpi.csv')

Finalmente se guarda el dataset final con el ['Total de vacunados y fallecidos por año y semana epidemiológica'](https://github.com/xxotto/covid19-peru/blob/main/Data/TOTAL_vacunados_y_fallecidos_x_semanaEpi.csv) de todo el Perú.

Disponible en el directorio [Data](https://github.com/xxotto/covid19-peru/tree/main/Data).